In [ ]:
# install psycopg2 so that connection to SaverLife database can be made


#import sys
#!{sys.executable} -m pip install psycopg2

import psycopg2

In [ ]:
# Establish connection to Saverlife database

conn = psycopg2.connect(
    host="REMOVED AS SPECIFIED UNDER OUR NDA",
    port=0000,
    database="REMOVED AS SPECIFIED UNDER OUR NDA",
    user="REMOVED AS SPECIFIED UNDER OUR NDA",
    password="REMOVED AS SPECIFIED UNDER OUR NDA")

In [ ]:
# Pull monthly spending, income, and transaction data for each user with a checking account. Pulls from
# Plaid Main Transactions first.

import pandas as pd

ts_pmt = """

select distinct c.user_id, e.name as campaign_name, 
                concat(extract(year from date),'_',extract(month from date)) as year_month,
                sum(case when amount_cents<0 then amount_cents else 0 END) as income,
                sum(case when amount_cents>0 then amount_cents else 0 END) as spending,
                sum(case when amount_cents>0 then 1 else 0 END) as spend_trans            
    from plaid_main_transactions as a
        inner join bank_accounts as b on a.bank_account_id=b.id
            inner join plaid_financial_authentications as c on b.plaid_financial_authentication_id=c.id
                left join marketing_conversions as d on c.user_id=d.user_id
                    left join marketing_partners as e on d.marketing_partner_id=e.id
                where extract(year from date) >= 2019 and account_subtype = 'checking' and category_id != '21001000'
                      and abs(amount_cents)<1000000 and c.is_current and c.state = 'connected'
                    group by c.user_id, e.name, year_month

"""

ts_pmt_df = pd.read_sql(ts_pmt, con=conn)


In [ ]:
# Pull monthly spending, income, and transaction data for each user with a checking account. Pulls from
# Plaid Auxiliary Transactions, but all features are otherwise the same as the above block.

ts_pat = """

select distinct c.user_id, e.name as campaign_name,
                concat(extract(year from date),'_',extract(month from date)) as year_month,
                sum(case when amount_cents<0 then amount_cents else 0 END) as income,
                sum(case when amount_cents>0 then amount_cents else 0 END) as spending,
                sum(case when amount_cents>0 then 1 else 0 END) as spend_trans            
    from plaid_auxiliary_transactions as a
        inner join bank_accounts as b on a.bank_account_id=b.id
            inner join plaid_financial_authentications as c on b.plaid_financial_authentication_id=c.id
                left join marketing_conversions as d on c.user_id=d.user_id
                    left join marketing_partners as e on d.marketing_partner_id=e.id
                where extract(year from date) >= 2019 and account_subtype = 'checking' and category_id != '21001000'
                      and abs(amount_cents)<1000000 and c.is_current and c.state = 'connected'
                    group by c.user_id, e.name, year_month

"""

ts_pat_df = pd.read_sql(ts_pat, con=conn)


In [ ]:
# Create daily account balances for every checking account in Plaid Main Transactions

balances_pmt = """

with recursive plaid_financial_authentications_days as (
 select pfa.user_id,
  pfa.id,
  pfa.created_at::date,
  pfa.created_at::date as date
 from plaid_financial_authentications as pfa
 where pfa.is_current
  and pfa.state = 'connected'
 union all
 select pfad.user_id,
  pfad.id,
  pfad.created_at,
  (pfad.date + interval '1 day')::date
 from plaid_financial_authentications_days as pfad
 where pfad.date < now()::date
)
select pfad.user_id,
 pfad.date, concat(extract(year from  pfad.date),'_',extract(month from  pfad.date)) as year_month,
 ba.current_balance_cents
  + coalesce(sum(pmt.amount_cents) over (
    partition by pfad.user_id
    order by pfad.date desc
  ), 0) as daily_balance
from plaid_financial_authentications_days as pfad
 join bank_accounts as ba on pfad.id = ba.plaid_financial_authentication_id
  and ba.main_saving
 left join plaid_main_transactions as pmt on ba.id = pmt.bank_account_id
  and pfad.date = pmt.date::date
    where ba.account_subtype = 'checking'
        order by pfad.user_id, pfad.date

"""

balance_pmt_df = pd.read_sql(balances_pmt, con=conn)


In [ ]:
# Create daily account balances for every checking account in Plaid Auxiliary Transactions

balances_pat = """

with recursive plaid_financial_authentications_days as (
 select pfa.user_id,
  pfa.id,
  pfa.created_at::date,
  pfa.created_at::date as date
 from plaid_financial_authentications as pfa
 where pfa.is_current
  and pfa.state = 'connected'
 union all
 select pfad.user_id,
  pfad.id,
  pfad.created_at,
  (pfad.date + interval '1 day')::date
 from plaid_financial_authentications_days as pfad
 where pfad.date < now()::date
)
select pfad.user_id,
 pfad.date, concat(extract(year from  pfad.date),'_',extract(month from  pfad.date)) as year_month,
 ba.current_balance_cents
  + coalesce(sum(pmt.amount_cents) over (
    partition by pfad.user_id
    order by pfad.date desc
  ), 0) as daily_balance
from plaid_financial_authentications_days as pfad
 join bank_accounts as ba on pfad.id = ba.plaid_financial_authentication_id
  and ba.main_saving='False'
 left join plaid_auxiliary_transactions as pmt on ba.id = pmt.bank_account_id
  and pfad.date = pmt.date::date
    where ba.account_subtype = 'checking'
        order by pfad.user_id, pfad.date

"""

balance_pat_df = pd.read_sql(balances_pat, con=conn)


In [ ]:
# Limit daily account balance dataframes to the first value in each month

balance_pmt_df['first'] = (balance_pmt_df.groupby(["user_id","year_month"]).cumcount() == 0).astype(int)

balance_pmt_ym = balance_pmt_df[balance_pmt_df['first'] == 1]

balance_pat_df['first'] = (balance_pat_df.groupby(["user_id","year_month"]).cumcount() == 0).astype(int)

balance_pat_ym = balance_pat_df[balance_pat_df['first'] == 1]


In [ ]:
# Pull the number of interactions that users have with the Saverlife website in each month

amp = """

select distinct b.id as user_id, concat(extract(year from event_time),'_',extract(month from event_time)) as year_month,
                count(a.user_id) as events
    from amplitude_events as a
        inner join users as b on a.user_id = b.amplitude_id
            group by b.id, year_month

"""

amp_df = pd.read_sql(amp, con=conn)


In [ ]:
# Merge monthly spending/income dataframe with monthly account balance dataframe

ts_pmt_df2 = pd.merge(ts_pmt_df, balance_pmt_ym, on=['user_id','year_month'], how='inner')

# Determing number of months that users' accounts are active

ts_pmt_df2['count'] = ts_pmt_df2['user_id'].map(ts_pmt_df2['user_id'].value_counts())

# Limit to users with at least 4 months of activity

ts_pmt_df3=ts_pmt_df2[ts_pmt_df2['count']>=4]

# Perform same steps with auxiliary accounts - merge monthly spending/income dataframe with monthly account
# balance dataframe

ts_pat_df2 = pd.merge(ts_pat_df, balance_pat_ym, on=['user_id','year_month'], how='inner')

# Determing number of months that users' accounts are active

ts_pat_df2['count'] = ts_pat_df2['user_id'].map(ts_pat_df2['user_id'].value_counts())

# Limit to users with at least 4 months of activity

ts_pat_df3=ts_pat_df2[ts_pat_df2['count']>=4]

# Append auxiliary accounts to main accounts

ts_all = ts_pmt_df3.append(ts_pat_df3)

# Drop and rename columns

ts_all2=ts_all.drop(columns=['date', 'first', 'count'])

ts_all2.rename(columns = {'daily_balance':'balance'}, inplace = True)


In [ ]:
# Merge spending/account balance dataframe with user interactions dataframe

ts_all3 = pd.merge(ts_all2, amp_df, on=['user_id','year_month'], how='left')


In [ ]:
#Change file location to save csv in desired location

ts_all3.to_csv(r'', index=False)